## Download the dataset:

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-24 13:50:33--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt.1'

input.txt.1         100%[===================>]   1.06M  5.67MB/s    in 0.2s    

2023-09-24 13:50:33 (5.67 MB/s) - 'input.txt.1' saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# The dataset:

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
''' First 500 characters of dataset '''
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [13]:
''' List all characters in dataset '''
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Characters: ",''.join(chars))
print("Amount of characters:", vocab_size)

Characters:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Amount of characters: 65


In [15]:
''' Create a mapping from characters to integers '''
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("Hello world")))


[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
Hello world


In [17]:
''' tokenize the dataset '''
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [19]:
''' split the data into train and validation sets '''
split = int(0.9*len(data)) # 90%, 10%
train_data = data[:split] # 90%
val_data = data[split:] # 10%

In [35]:
''' Examples of inputs and outputs'''
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is {target}.")

when input is tensor([18]), the target is 47.
when input is tensor([18, 47]), the target is 56.
when input is tensor([18, 47, 56]), the target is 57.
when input is tensor([18, 47, 56, 57]), the target is 58.
when input is tensor([18, 47, 56, 57, 58]), the target is 1.
when input is tensor([18, 47, 56, 57, 58,  1]), the target is 15.
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47.
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58.


In [40]:
''' Creating bathes with examples '''
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(dataset):
    data = train_data if dataset== "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

x_batch, y_batch = get_batch("train")
print('inputs:')
print(x_batch.shape)
print(x_batch)
print('outputs:')
print(y_batch.shape)
print(y_batch)

print('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = x_batch[b, :t+1]
        target = y_batch[b, t]
        print(f"When input is {context.tolist()}, the target is {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
When input is [24], the target is 43
When input is [24, 43], the target is 58
When input is [24, 43, 58], the target is 5
When input is [24, 43, 58, 5], the target is 57
When input is [24, 43, 58, 5, 57], the target is 1
When input is [24, 43, 58, 5, 57, 1], the target is 46
When input is [24, 43, 58, 5, 57, 1, 46], the target is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39
When input is [44], the target is 53
When input is [44, 53], the target is 56
When input is [44, 53, 56], the target is 1
When input is [44, 53, 56, 1], the target is 58
When input is [44, 53, 56, 1, 58]

# Creating the model:

In [55]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B ,T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # get predictions
            logits = logits[:,-1,:] # focus on last time step (B, C)
            probs = F.softmax(logits, dim=-1) # get probabilities (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # Sample from distribution
            idx = torch.cat((idx, idx_next), dim=1) # append sample to running sequence (B, T+1)
        return idx


model = BigramLanguageModel(vocab_size)
logits, loss = model(x_batch, y_batch)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=500)[0].tolist()))

torch.Size([256, 65])
tensor(4.7336, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJt-wBpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUiQQY:qFINTOBNLI,&oTigq z.c:Cq,SDXzetn3XVjX-YBcHAUhk&PHdhcOb
nhJ?FJU?pRiOLQeUN!BxjPLiq-GJdUV'hsnla!murI!IM?SPNPq?VgC'R
pD3cLv-bxn-tL!upg
SZ!Uvdg CtxtT?hsiW:XxKIiPlagHIsr'zKSVxza?GlDWObPmRJgrIAcmspmZ&viCKot:u3qYXA:rZgv f:3Q-oiwUzqh'Z!I'zRS3SP rVchSFUIdd q?sPJpUdhMCK$VXXevXJFMl,i
YxA:gWId,EXR,iMC,$?srV$VztRwb?KpgUWFjR$zChOLm;JrDnDph
LBj,KZxJa


# Training the model:

In [46]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [50]:
batch_size = 32
for steps in range(10000):
    x_batch, y_batch = get_batch('train')

    logits, loss = model(x_batch,y_batch)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5691680908203125


In [54]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


kntr I se y ce sech is arsenased tthentsman lemout:
I
We;


NGle, LAndr!
Fimau f,
QUTirourthithak, malit I myon Ispoupe mom;
H:
hades cks thaker de benlleall in
OUETCHUETHare t VABUSAmie irrowaks

Hadianse t ral-eam, are, RI ar?
CETh thosthe my, th curous ver h ch kityonoyoveeld,-w dsupu presoto-tll GLombur g!
EOREEDe, mer peanthedey ninches lom touds ag wer, g, thowlassue a'd blld he
Weenbe d: andaly mmed, Gorid, capue; ounertheyart ak:
hinod tord tty t s y,
Toroittchaceras dy, mpe g.
S:
S: ths


In [56]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [59]:
''' We want to average the information which came before the current character '''
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [64]:
''' A much more efficient implementation of the previous cell'''
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [67]:
''' A third way of implementing it using softmax! '''
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow,xbow3)

True

In [66]:
print(xbow[0])
print(xbow2[0])

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
